## Modelo de datos de empleados

#### Este modelo de datos es un modelo introductorio. Representa los detalles básicos de un empleado, como un alias único, nombre, apellido, designación, gerente y habilidades.

Este modelo de datos representa algunas técnicas como el manejo de atributos complejos, como tener más de una habilidad. Este modelo es también un ejemplo de relación uno a muchos a través del gerente y sus empleados de informes que se ha logrado mediante el índice secundario DirectReports.

Los patrones de acceso que este modelo de datos facilita son:

* Recuperación de un registro de empleado utilizando el alias de inicio de sesión del empleado, facilitada por una tabla llamada Employee.

* Buscar empleados por nombre, facilitado por el índice secundario global de la tabla Empleado llamado Name.

* Recuperación de todos los informes directos de un gerente utilizando el alias de inicio de sesión del gerente, facilitada por el índice secundario global de la tabla Empleado llamado DirectReports.

TABLE:
<br>
![image.png](images/employees01.png)

GSI: Name
<br>
![image.png](images/employees02.png)

GSI: DirectReports
<br>
![image.png](images/employees03.png)

In [1]:
import pandas as pd
from spdynamodb import DynamoTable
from time import sleep
import json
from decimal import Decimal

In [2]:
dt = DynamoTable()
try:
    dt.select_table('Employees')
    print(dt)
except:
    dt.create_table(
        table_name='Employees',
        partition_key='LoginAlias',
        partition_key_type='S',
        provisioned=True,
)

Table created successfully!


In [3]:
dt.load_json("employees.json")

In [4]:
dt.bill_mode

'PROVISIONED'

**Los índices secundarios globales tienen capacidad de lectura y escritura independiente de la tabla subyacente.**
<br>
Si no se selecciona PAY_PER_REQUEST al crear la tabla

In [5]:
dt.create_global_secondary_index(
    att_name="FirstName",
    att_type="S",
    sort_index="LastName",
    sort_type="S",
    i_name="NameIndex",
    i_rcu=1,
    i_wcu=1
)

In [6]:
status = dt.check_status_gsi()
while status == 'CREATING':
    status = dt.check_status_gsi()
    sleep(30)
print("Global secondary index created.")

In [ ]:
dt.create_global_secondary_index(
    att_name="ManagerLoginAlias",
    att_type="S",
    i_name="DirectReportsIndex",
    proj_type=["FirstName", "LastName"],
    i_rcu=10,
    i_wcu=10
)

In [ ]:
status = dt.check_status_gsi()
while status == 'CREATING':
    status = dt.check_status_gsi()
    sleep(30)
print("Global secondary index created.")

In [11]:
dt.list_gsi

['NameIndex']

In [12]:
dt.check_status_gsi()

'ACTIVE'

In [8]:
dt.query_items("janed", to_pandas=True)

,LoginAlias,ManagerLoginAlias,FirstName,LastName,Skills
0,janed,marthar,Jane,Doe,[software]
